In [10]:
from CB import *
from UserCF import *

In [11]:
# 输出推荐给该用户的节目列表
def printRecommendItems(recommend_items_sorted, max_num):
    '''
    recommend_items_sorted：按感兴趣程度降序排列的推荐节目集，[[节目一, 该用户对节目一的感兴趣程度],[...], ...]
    max_num：最多输出的推荐节目数
    '''
    count = 0
    for item, degree in recommend_items_sorted:
        print("节目名：%s， 推荐指数：%f" % (item, degree))
        count += 1
        if count == max_num:
            break

In [13]:
# 主程序
if __name__ == '__main__':
    
    '''UserCF'''
    
    # 所有用户名
    all_users_names = ['A', 'B', 'C']
    
    # 按指定顺序排列的所有标签（类型）
    all_labels = ['教育', '戏曲', '悬疑', '科幻', '惊悚', '动作', '资讯', '武侠', '剧情', '警匪', '生活', '军事', '言情', '体育', '冒险', '纪实',
                  '少儿教育', '少儿', '综艺', '古装', '搞笑', '广告']
    labels_num = len(all_labels)
        
    # 所有用户对其看过的节目的评分矩阵
    df1 = pd.read_excel("../data/所有用户对其看过的节目的评分矩阵.xlsx")
    (m1, n1) = df1.shape
    data_array1 = np.array(df1.iloc[:m1 + 1, 1:])  # [[0.1804 0.042 0.11  0.07  0.19  0.56  0.14  0.3  0.32 0, ...], [...] ...]
    items_users_saw_names1 = df1.columns[1:].tolist()  # ['大军师司马懿之军师联盟', '非诚勿扰', '记忆大师', ...]
    
    # 所有用户的观看信息（包含隐性评分信息），{用户一:[['节目一', 3.2], ['节目四', 0.2], ['节目八', 6.5]], 用户二: ... }
    users_dict = createUsersDict(df1)
    # 所有节目被哪些用户观看的字典，{节目一: [用户一, 用户三], 节目二: ... }
    items_dict = createItemsDict(df1)    
    
    '''CB'''
    
    # 所有用户看过的节目及所属类型的01矩阵
    df2 = pd.read_excel("../data/所有用户看过的节目及所属类型的01矩阵.xlsx")
    (m2, n2) = df2.shape
    data_array2 = np.array(df2.iloc[:m2 + 1, 1:])  # [[0, 0, 0, 0, 0, 0, 0, 0, 1, ...], [...] ...]
    items_users_saw_names2 = np.array(df2.iloc[:m2 + 1, 0]).tolist()  # ['大军师司马懿之军师联盟', '非诚勿扰', '记忆大师', ...]
    
    # 为用户看过的节目建立节目画像items_users_saw_profiles
    items_users_saw_profiles = createItemsProfiles(data_array2, all_labels, items_users_saw_names2)

    # 建立用户画像users_profiles和用户看过的节目集items_users_saw
    (users_profiles, items_users_saw) = createUsersProfiles(data_array1, all_users_names, items_users_saw_names1, all_labels, items_users_saw_profiles)

    # 备选推荐节目集及所属类型01矩阵
    df3 = pd.read_excel("../data/备选推荐节目集及所属类型01矩阵.xlsx")
    (m3, n3) = df3.shape
    data_array3 = np.array(df3.iloc[:m3 + 1, 1:])  # [[0, 0, 0, 0, 0, 1, ...], [...] ...]
    items_to_be_recommended_names = np.array(df3.iloc[:m3 + 1, 0]).tolist()  # ['绑架者', '10月29日探索·发现', '银河护卫队', ...]
    
    # 为备选推荐节目集建立节目画像
    items_to_be_recommended_profiles = createItemsProfiles(data_array3, all_labels, items_to_be_recommended_names)

    '''两种推荐算法后融合，也就是将两种推荐算法对某个用户分别产生的两个推荐节目集按不同比例混合，得出最后的对该用户的推荐结果
    对于每个用户推荐topN个节目,在两种推荐算法产生的推荐集中分别选取比例为w1和w2的推荐结果,CB占w1, userCF占w2
    w1 + w2 = 1 且 w1 * topN + w2 * topN = topN'''

    topN = 5

    w1 = 0.7
    w2 = 0.3

    # 从CB的推荐集中选出前topW1项
    topW1 = int(w1 * topN)

    # 从userCF的推荐集中选出前topW2项
    topW2 = topN-topW1

    for user in all_users_names:

        # 对于用户的最终混合推荐节目集
        recommend_items = []
        
        # CB，[[节目名, 该节目与该用户画像的相似度], ...]
        recommend_items1 = contentBased(users_profiles[user], items_to_be_recommended_profiles, items_to_be_recommended_names, all_labels, items_users_saw[user])
        len1 = len(recommend_items1)

        if len1 <= topW1:
            recommend_items = recommend_items + recommend_items1
        else:
            recommend_items = recommend_items + recommend_items1[:topW1]

        # userCF，[[节目名, 该用户对该节目的感兴趣程度],...]
        recommend_items2 = userCF(user, users_dict, items_dict, 2, items_to_be_recommended_names)
        len2 = len(recommend_items2)

        if len2 <= topW2:
            recommend_items = recommend_items + recommend_items2
        else:
            recommend_items = recommend_items + recommend_items2[:topW2]

        # 将推荐结果按推荐指数降序排序
        recommend_items.sort(key=lambda item: item[1], reverse=True)

        print("对于用户 %s 的推荐节目如下" % user)
        printRecommendItems(recommend_items, 5)
        print()

对于用户 A 的推荐节目如下
节目名：老友记第五季， 推荐指数：1.000000
节目名：成长边缘， 推荐指数：1.000000
节目名：原谅他77次， 推荐指数：0.847216
节目名：合约男女， 推荐指数：0.847216
节目名：英雄出少年， 推荐指数：0.711127

对于用户 B 的推荐节目如下
节目名：幻觉围城， 推荐指数：0.622305
节目名：分裂， 推荐指数：0.483290
节目名：糖衣陷阱， 推荐指数：0.420428
节目名：非诚勿扰， 推荐指数：-0.293065
节目名：老友记第五季， 推荐指数：-0.547594

对于用户 C 的推荐节目如下
节目名：非诚勿扰， 推荐指数：1.000000
节目名：我是谁的宝贝， 推荐指数：0.106075
节目名：危楼愚夫， 推荐指数：0.106075
节目名：明月几时有， 推荐指数：0.106075
节目名：致命礼物， 推荐指数：-0.547594

